In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 19 16:41:39 2022

@author: Sushmith
"""

import numpy as np

batch_size = 100

def get_batch_nmd(P, C, N):
    h = N // 2
    assert P.shape == C.shape
    CropP = P[:, h:-h, h:-h, :]
    Dists = np.empty((N*N, P.shape[0], P.shape[1]-N+1, P.shape[2]-N+1))
    p = 0
    for i in range(-h, h+1):
        for j in range(-h, h+1):
            i0, i1 = i+h, C.shape[1]+i-h
            j0, j1 = j+h, C.shape[2]+j-h
            CropC = C[:, i0:i1, j0:j1, :]
            Dists[p, :, :, :] = np.sum(abs(CropP-CropC), axis=3)
            p = p + 1
    NMD3D = np.min(Dists, axis=0)
    NMD = np.empty((NMD3D.shape[0],NMD3D.shape[1]*NMD3D.shape[2]))
    for p in range(NMD3D.shape[0]):
        NMD[p, :] = NMD3D[p].flatten()
    return NMD

def get_nmd(P, C, N):
    assert P.shape == C.shape
    NMD = np.empty((P.shape[0],(P.shape[1]-N+1)*(P.shape[2]-N+1)))
    for batch in range(0, P.shape[0], batch_size):
        size = min(batch_size, P.shape[0]-batch)
        NMD[batch:batch+size, :] = get_batch_nmd(P[batch:batch+size, :, :, :], C[batch:batch+size, :, :, :], N)
    return NMD





from imageio import imread
import scipy.io as sio
from os.path import join

def load_txt_pairs(filename):
    metafile = open(filename, 'r')
    metadata = metafile.read()
    metafile.close()
    metalines = [line for line in metadata.split('\n') if len(line)>0]
    str_pairs = [metaline.split(' ') for metaline in metalines]
    pairs = [[int(pair[0]), int(pair[1]), pair[2], pair[3]] for pair in str_pairs]
    return pairs

def load_mat_pairs(filename):
    meta = sio.loadmat(filename)
    pairs = []
    for p in meta['pairs']:
        pairs.append([p[0][0][0], p[1][0][0], p[2][0], p[3][0]])
    return pairs

def get_image_dirs(rootdir, dataset, subset):
    PrefixToDir={'fd':'father-dau','fs':'father-son','md':'mother-dau','ms':'mother-son'}
    if dataset[:10]=='KinFaceW-I':
        same_dir = join(rootdir, dataset, 'images', PrefixToDir[subset])
        return (same_dir, same_dir)
    if dataset=='CornellKinFace':
        parents_dir = join(rootdir, dataset, 'Parents')
        children_dir = join(rootdir, dataset, 'Children')
        return (parents_dir, children_dir)
    return None

def load_pairs(rootdir, dataset, subset):
    if dataset=='KinFaceW-I' or dataset=='KinFaceW-II':
        return load_mat_pairs(join(rootdir, dataset, 'meta_data', subset + '_pairs.mat'))
    if dataset=='CornellKinFace':
        return load_txt_pairs(join(rootdir, dataset, 'cornell-meta.txt'))
    return None

def load_fold(rootdir, dataset, subset, fold):
    pairs = load_pairs(rootdir, dataset, subset)
    image_dirs = get_image_dirs(rootdir, dataset, subset)
    P0, C0, K0 = [], [], []
    P1, C1, K1 = [], [], []
    for p in pairs:
        pImg = imread(join(image_dirs[0], p[2])) / 255
        cImg = imread(join(image_dirs[1], p[3])) / 255
        if p[0] == fold:
            P1.append(pImg)
            C1.append(cImg)
            K1.append(p[1])
        else:
            P0.append(pImg)
            C0.append(cImg)
            K0.append(p[1])
    return ((np.array(P0), np.array(C0), np.array(K0)), (np.array(P1), np.array(C1), np.array(K1)))





from sklearn import svm
from sklearn import ensemble

rootdir = 'C:/Users/Bhargava/Desktop/kinface'

datasets = [
            ('KinFaceW-I', 'fs', 5),
            ('KinFaceW-I', 'fd', 5),
            ('KinFaceW-I', 'ms', 5),
            ('KinFaceW-I', 'md', 5),
            ('KinFaceW-II', 'fs', 5),
            ('KinFaceW-II', 'fd', 5),
            ('KinFaceW-II', 'ms', 5),
            ('KinFaceW-II', 'md', 5)]

nu_values = np.arange(0.05, 1.0, 0.05)
max_depth_values = [max_depth for max_depth in range(10, 16)]
n_estimators_values = [n_estimators for n_estimators in range(1, 6)]
for (dataset, kinship, n_fold) in datasets:
    print(dataset, kinship)
    svm_accu_grid = np.zeros(len(nu_values), dtype=np.float32)
    rdf_accu_grid = np.zeros((len(max_depth_values), len(n_estimators_values)), dtype=np.float32)
    for fold in range(n_fold):
        ((P0, C0, K0), (P1, C1, K1)) = load_fold(rootdir, dataset, kinship, fold+1)

        NMD0 = get_nmd(P0, C0, 15)
        NMD1 = get_nmd(P1, C1, 15)


        for i, nu in enumerate(nu_values):
            model = svm.NuSVC(nu=nu, kernel="rbf", verbose=False, gamma="auto")
            model.fit(NMD0, K0)
            accuracy = model.score(NMD1, K1)
            svm_accu_grid[i] = svm_accu_grid[i] + accuracy
            
        for i, max_depth in enumerate(max_depth_values):
            for j, n_estimators in enumerate(n_estimators_values):
                best_accu = 0.0
                for _ in range(5):
                    model = ensemble.RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=1)
                    model.fit(NMD0, K0)
                    accuracy = model.score(NMD1, K1)
                    if accuracy > best_accu:
                        best_accu = accuracy
                rdf_accu_grid[i, j] = rdf_accu_grid[i, j] + best_accu

    best_nu = 0
    best_accu = 0
    for i, nu in enumerate(nu_values):
        if svm_accu_grid[i] > best_accu:
            best_accu = svm_accu_grid[i]
            best_nu = nu
    
    print('SVM: %.02f%% [nu: %g]'%(100*best_accu/n_fold, best_nu))
    
    best_d = 0
    best_n = 0
    best_a = 0
    for i, max_depth in enumerate(max_depth_values):
        for j, n_estimators in enumerate(n_estimators_values):
            if rdf_accu_grid[i, j]>best_a:
                best_d = max_depth
                best_n = n_estimators
                best_a = rdf_accu_grid[i, j]
    
    print('RandomForest: %.02f%% [max_depth: %d] [n_estimators: %d]'%(100*best_a/n_fold, best_d, best_n))